# From chatgpt.com

## 1
Podrías hacer un código python que lance un query SQL a una base de datos SQL Server y que el resultado de dicha query la guarde como un archivo csv separado por ';' codificado utf-8 con BOM

Claro, aquí tienes un ejemplo de código Python que se conecta a una base de datos SQL Server, ejecuta una consulta SQL, y guarda el resultado en un archivo CSV con separador `;` y codificación `utf-8-sig` (que incluye el BOM):

### ✅ Requisitos previos

Necesitas instalar el paquete `pyodbc` si no lo tienes:

```bash
pip install pyodbc
```

---

### 💻 Código Python

```python
import pyodbc
import csv

# Parámetros de conexión (ajusta según tu configuración)
server = 'NOMBRE_DEL_SERVIDOR'
database = 'NOMBRE_DE_LA_BASE_DE_DATOS'
username = 'USUARIO'
password = 'CONTRASEÑA'

# Query SQL que quieres ejecutar
sql_query = 'SELECT * FROM tu_tabla'

# Nombre del archivo CSV de salida
output_file = 'resultado.csv'

# Establece la conexión
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
)

try:
    with pyodbc.connect(conn_str) as conn:
        cursor = conn.cursor()
        cursor.execute(sql_query)

        # Obtener nombres de columnas
        columns = [column[0] for column in cursor.description]

        # Escribir a CSV con separador ';' y codificación UTF-8 con BOM
        with open(output_file, mode='w', newline='', encoding='utf-8-sig') as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow(columns)  # Escribe encabezados
            for row in cursor:
                writer.writerow(row)

    print(f"Archivo CSV guardado exitosamente como '{output_file}'.")

except Exception as e:
    print("Ocurrió un error:", e)
```

---

¿Te gustaría que te ayudara a generar una versión parametrizable desde línea de comandos o con una interfaz simple?
- Afirmativo

Perfecto. A continuación, te presento una **versión mejorada y parametrizable desde línea de comandos**. Esta versión permite pasar los parámetros como argumentos, ideal para automatización o uso repetido.

---

### 💻 Código Python (parametrizable vía CLI)

Guarda esto como `export_sql_to_csv.py`:

```python
import argparse
import pyodbc
import csv
import sys

def export_query_to_csv(server, database, username, password, query, output_file):
    conn_str = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        f'SERVER={server};DATABASE={database};UID={username};PWD={password}'
    )

    try:
        with pyodbc.connect(conn_str) as conn:
            cursor = conn.cursor()
            cursor.execute(query)

            # Obtener nombres de columnas
            columns = [column[0] for column in cursor.description]

            # Escribir resultados en CSV
            with open(output_file, mode='w', newline='', encoding='utf-8-sig') as f:
                writer = csv.writer(f, delimiter=';')
                writer.writerow(columns)
                for row in cursor:
                    writer.writerow(row)

        print(f"✅ Archivo CSV guardado como '{output_file}'.")

    except Exception as e:
        print(f"❌ Error: {e}")
        sys.exit(1)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Exporta el resultado de una consulta SQL Server a un archivo CSV.')

    parser.add_argument('--server', required=True, help='Nombre del servidor SQL Server')
    parser.add_argument('--database', required=True, help='Nombre de la base de datos')
    parser.add_argument('--username', required=True, help='Usuario de SQL Server')
    parser.add_argument('--password', required=True, help='Contraseña del usuario')
    parser.add_argument('--query', required=True, help='Consulta SQL que se va a ejecutar (en una sola línea o entre comillas)')
    parser.add_argument('--output', required=True, help='Ruta del archivo CSV de salida')

    args = parser.parse_args()

    export_query_to_csv(
        server=args.server,
        database=args.database,
        username=args.username,
        password=args.password,
        query=args.query,
        output_file=args.output
    )
```

---

### 🧪 Ejemplo de uso desde la terminal

```bash
python export_sql_to_csv.py \
  --server localhost \
  --database MiBase \
  --username sa \
  --password MiContraseña123 \
  --query "SELECT * FROM empleados" \
  --output empleados.csv
```

---

¿Quieres que también soporte autenticación de Windows (integrada)?
